# Post Processing API Queries
In most cases you will need to processes the raw API response to conduct analysis specific to your use case. This notebook gives some examples of common post processing you may need to do.

## Table of Contents
1. [Converting to different file formats](#file-formats)
    1. [Shapefile](#shapefile)
    2. [Geojson](#geojson)
    3. [Excel spreadsheet (csv)](#spreadsheet)
2. [Collapse geometry to point](#collapse)
3. [Combine with census data](#census)
4. [Convert postcode to lat lon](#postcode)
5. [Add nearest building to greenspace](#nearest)

# Converting to different file formats<a name="file-formats" />
The OS API Wrapper returns queries as a python dictionary in geojson format. The following examples show how to convert the response geojson into a GeoDataFrame and save as a geojson and shapefile.

If you do not need to do any geospatial analysis, then you could save the response as a csv. A way of doing this is also shown.

The code block immediately below will get the raw response.

In [ ]:
from OSAPIWrapper import Extent
from OSAPIWrapper.FeaturesAPI import FeaturesAPI
from os import environ

# Set your API key and crs
key = environ.get("OS_API_KEY")
crs = "EPSG:27700"

# Define a bounding box extent for the reigion you want
extent = Extent.from_bbox((600000, 310200, 600900, 310900), crs)

# Choose your product
product = "zoomstack_local_buildings"
features_api = FeaturesAPI(key, product, extent)

# Make the API call
raw_response = features_api.query(limit=350)

## Shapefile<a name="shapefile" />
We will first convert the raw_response into a GeoDataFrame, which makes it easier to manipulate geospatial data.

In [ ]:
import geopandas as gpd

# Use the from_features() method to create a GeoDataFrame from a dictionary
gdf = gpd.GeoDataFrame.from_features(raw_response['features'], crs=crs)

# View the GeoDataFrame
gdf.head()

In [ ]:
# Save the geodataframe as a shapefile
gdf.to_file("zoomsack_buildings.shp")

## Geojson<a name="geojson" />

In [ ]:
# Save the geodataframe as a geojson
gdf.to_file("zoomsack_buildings.geojson", driver="GeoJSON")

## Spreadsheet<a name="spreadsheet" />
They may be times where you are not interest in the geospatial aspect of the data and would therefore like to view the data in a spreadsheet. You could do this by converting the GeoDataFrame into a normal pandas DataFrame and then saving as a csv.

Note that we will drop the `geometry` column in this example but this step is optional. If you don't drop the `geometry` column it will be saved as a string in the csv. The string representation of geometry is know as Well Known Text (WKT)

In [ ]:
import pandas as pd


df = pd.DataFrame(gdf.drop(columns=["geometry"]))
df.to_csv("zoomstack_buildings.csv")

# Collapse geometry to point<a name="collapse" />
Sometimes you may need to do geospatial analysis but may not require a polygon or linestring. It can therefore be useful to collapse polygons and lines into a single point. This will reduce your file size and also speed up certain geospatial calculations such as nearest neighbour.

This example will use the same raw response we got from the [Converting to different file formats](#file-formats) section.

In [ ]:
import geopandas as gpd
from OSAPIWrapper import Extent
from OSAPIWrapper.FeaturesAPI import FeaturesAPI
from os import environ


# Set your API key and crs
key = environ.get("OS_API_KEY")
crs = "EPSG:27700"

# Define a bounding box extent for the reigion you want
extent = Extent.from_bbox((600000, 310200, 600900, 310900), crs)

# Choose your product
product = "zoomstack_local_buildings"
features_api = FeaturesAPI(key, product, extent)

# Make the API call
raw_response = features_api.query(limit=350)

# Use the from_features() method to create a GeoDataFrame from the raw response
gdf = gpd.GeoDataFrame.from_features(raw_response['features'], crs=crs)

# View the geometry type before collaposing to point
print(gdf.geometry.iloc[0:5], end="\n\n")

# GeoDataFrames have a built in attribute called "centroid" which calculates the middle point of the geometry column
# Set the geometry column equal to this attribute
gdf.geometry = gdf.centroid

# View the geometry type after collaposing to point
print(gdf.geometry.iloc[0:5])

# Combine with census data<a name="census" />
Using the API wrapper, you can combine data provided in the standard ONS census geographies with Ordnance Survey data. This section will assume you have a working knowledge of the data supplied by ONS.

In the following example, suppose you have a dataset which contains the codes of some LSOAs in the City of London. We will extract buildings for each LSOA and then join the two datasets on the LSOA code.

In [ ]:
import pandas as pd


# Create a sample dataframe
lsoa_codes = ["E01000001", "E01000002", "E01000003", "E01000005", "E01032739"]
os_maps_per_household = [1, 8, 2, 3, 2]
anuual_miles_hiked_per_household = [101, 428, 42, 23, 12]

ons_df = pd.DataFrame({"ons_code": lsoa_codes,
                       "os_maps": os_maps_per_household,
                       "anuual_miles_hiked": anuual_miles_hiked_per_household,
                       })
ons_df

In [ ]:
import geopandas as gpd
from OSAPIWrapper import Extent
from OSAPIWrapper.FeaturesAPI import FeaturesAPI
from os import environ


# Set your API key and crs
key = environ.get("OS_API_KEY")
product = "zoomstack_local_buildings"

# Iterate through the ONS codes and extract local buildings in each LSOA
features = []
for ons_code in lsoa_codes:
    # Create an extent using the "from_ons_code()" method
    # Note that when using the "from_ons_code()" method the response will always have CRS 4326
    extent = Extent.from_ons_code(ons_code)
    
    # Make the query
    features_api = FeaturesAPI(key, product, extent)
    raw_response = features_api.query(limit=50)
    
    # Add the ons code to the response
    for building in raw_response["features"]:
        building["properties"]["ons_code"] = ons_code
    
    features.extend(raw_response["features"])
    crs = raw_response["crs"]
    

# Create geodataframe
gdf = gpd.GeoDataFrame.from_features(features, crs=crs)
gdf.head()

Next, we join this GeoDataFrame to the ons_df

Note that if you merge a GeoDataFrame with a DataFrame you will get a GeoDataFrame. Similarly if you merge a DataFrame with a GeoDataFrame, in that order, you will get a DataFrame.

In [ ]:
gdf.merge(ons_df, on="ons_code").head()

# Convert postcode to lat lon<a name="postcode" />
In this section we show how to reproject a GeoDataFrame to a different CRS.

In [ ]:
import geopandas as gpd
from OSAPIWrapper.PlacesAPI import PlacesAPI
from os import environ


key = environ.get("OS_API_KEY")
crs = "epsg:27700"
places_api = PlacesAPI(key)

# Query postcodes in BNG (27700)
postcodes = places_api.postcode("SO16 0AS", output_crs=crs)

# Convert response to GeoDataFrame
gdf = gpd.GeoDataFrame.from_features(postcodes["features"], crs=crs)

# Print geometry before projection
print("***Before reprojection***", gdf.geometry, sep="\n", end="\n\n")

# Reproject to WGS84 (4326)
gdf.to_crs("epsg:4326", inplace=True)

print("***After reprojection***", gdf.geometry, sep="\n")

# Add nearest building to greenspace<a name="nearest" />
Sometimes you may have two datasets and will want to know what the closest element is for each row in a dataset. For example, you might get the roads dataset and the buildings dataset and  then want to attribute each road with the buildings that are nearest to it.

Note that this example shows how to join features spatially but the datahub provides an API called [Linked Identifier](https://osdatahub.os.uk/docs/linkedIdentifiers/overview) which has the official linkage dataset. However, it is not supported by the OS API Wrapper.

This section will query the greenspace data and postcode data separately. Then for each greenspace, we will find all the postcodes that are closest to the greenspace.

In [ ]:
import geopandas as gpd
from OSAPIWrapper import Extent
from OSAPIWrapper.PlacesAPI import PlacesAPI
from OSAPIWrapper.FeaturesAPI import FeaturesAPI
from os import environ

key = environ.get("OS_API_KEY")
crs = "epsg:27700"
product = "greenspace_area"

extent = Extent.from_bbox((598000, 310200, 600900, 310900), crs)

places_api = PlacesAPI(key)
raw_places = places_api.query(extent, output_crs=crs)


features_api = FeaturesAPI(key, product, extent)
raw_greenspace = features_api.query(limit=50)

# Query data 
places_gdf = gpd.GeoDataFrame.from_features(raw_places["features"], crs=crs)
greenspace_gdf = gpd.GeoDataFrame.from_features(raw_greenspace["features"], crs=crs)

In [ ]:
def get_nearest_uprn(greenspace_geometry, places_gdf):
    # Calculate the distance between the input greenspace and all postcodes
    distances = places_gdf.distance(greenspace_geometry)
    
    # Order the distance with shortest first
    distances = distances.sort_values()
    
    # Get the index of the nearest UPRN
    nearest_uprn_index = distances.index[0]
    
    # Get the UPRN code
    nearest_uprn_code = places_gdf.loc[nearest_uprn_index].UPRN
    return nearest_uprn_code

# Add column for the nearest greenspace by applying the get_nearest_uprn() function
greenspace_gdf["nearest_uprn"] = greenspace_gdf.apply(lambda x: get_nearest_uprn(x.geometry, places_gdf), axis=1)

greenspace_gdf.head()